# 필수라이브러리 및 데이터 경로 정적으로 정의 밑 import

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
DATA_PATH='/kaggle/input/usa-housing/USA_Housing.csv'

분류가 아닌 가격 자체를 예측 하는 문제로써 정규화 함수들은 필요가없어보입니다.

In [ ]:
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
housing = pd.read_csv(DATA_PATH)

# Housing INFO

In [ ]:
housing.head()

숫자 풀들이 큼직큼직하게 큼 다들 numeric한 변수라 딱히 건들건 없어보이고 피쳐들의 특성만 보고 넘어가는 방법으로 하겠음 다만, 맨마지막 주소는 뭔가 쓸만한 정보들이 있어보이기도합니다

In [ ]:
housing.describe()

In [ ]:
housing.isnull().sum()

In [ ]:
sns.pairplot(housing)

In [ ]:
housing.hist(bins=50,figsize=(20,15))

독립변수들의 분포도인데 크게 모델학습에 있어서 영향을 줄만한 변수는 보이지않는 것 같습니다.

In [ ]:
import scipy as sp

cor_abs = abs(housing.corr(method='spearman')) 
cor_cols = cor_abs.nlargest(n=6, columns='Price').index 
# spearman coefficient matrix
cor = np.array(sp.stats.spearmanr(housing[cor_cols].values))[0] # 10 x 10
print(cor_cols.values)
plt.figure(figsize=(10,10))
sns.set(font_scale=1.25)
sns.heatmap(cor, fmt='.2f', annot=True, square=True , annot_kws={'size' : 8} ,xticklabels=cor_cols.values, yticklabels=cor_cols.values)

price과 correlation이 높은 column 6개 뽑아서 내림차순으로 히트맵을 그립니다. 

Address는 문자열이라 그런지 출력이 되지 않네요.
그리고 당연히 월세가 높은 곳이 땅값이 비싸서 그런지 income이 가장 높은 상관관계도를 보이고 있습니다. 

# Avg. Area Income

In [ ]:
housing["Avg. Area Income"].describe()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(housing["Avg. Area Income"],ax=ax)
plt.show()


In [ ]:
housing.plot(kind="scatter",x="Avg. Area Income",y="Price",alpha=0.2,cmap= plt.get_cmap("jet"),colorbar=True)

크게 심각한 이상치는 보이지않고 것 같습니다. 고대로 진행하겠습니다.

# Avg. Area House Age

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(housing["Avg. Area House Age"],ax=ax)
plt.show()



In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(20, 18))
ax = fig.add_subplot(111, projection='3d')

xs = housing['Avg. Area Income']
ys = housing['Avg. Area House Age']
zs = housing['Price']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, edgecolors='w')

ax.set_xlabel('Avg. Area Income')
ax.set_ylabel('Avg. Area House Age')
ax.set_zlabel('Price')
ax.scatter(xs, ys, zs, c = zs, s= 50, alpha=0.5, cmap=plt.cm.Greens)
#ax.view_init(60, 35)
plt.show()

 집 연세보단 우선 방세가 비싸면 집값이 비싸군요. 역시 강남 같은 대도시인가 아니면 고속도로에 경운기 다니는 촌구석인가 위치의 영향이 큰거 같습니다.

# Avg. Area Number of Rooms

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(housing["Avg. Area Number of Rooms"],ax=ax)
plt.show()

방의 갯수에 대한 분포도입니다.딱히 건드릴만한게 있어보이진않습니다 

# Avg. Area Number of Bedrooms

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(housing["Avg. Area Number of Bedrooms"],ax=ax)
plt.show()

침실 갯수에 대한 분포도인데 역시 건들거는 없어보입니다.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(20, 18))
ax = fig.add_subplot(111, projection='3d')

xs = housing['Avg. Area Number of Bedrooms']
ys = housing['Avg. Area Number of Rooms']
zs = housing['Price']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, edgecolors='w')

ax.set_xlabel('Number of Bedrooms')
ax.set_ylabel('Number of Rooms')
ax.set_zlabel('Price')
ax.scatter(xs, ys, zs, c = zs, s= 50, alpha=0.5, cmap=plt.cm.Greens)
#ax.view_init(60, 35)
plt.show()

침실이든 전체방의 갯수든 방이 항상 많다고 가격이 비싼건 아닙니다. 

방이 많은게 없는 것보다 좋기는 한데, 방이 많다고 무조건 비싼 집은 아니라는 이야기입니다.

# Area Population

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(housing["Area Population"],ax=ax)
plt.show()

오염도에 대한 분포도입니다. 스케일이 크긴한데 집값 예측이라 건들건 없어보입니다.

# Address

주소입니다. 초반부에 얘기했듯이 강남에 집을 지어서 파는거랑 경운기 굴러다니는 촌구석에 집 지어서 파는거랑 당현히 가격 차이가 납니다. 
저는 이 주소 데이터에서 전처리를 통해 그 특성에 대해서 알아 보도록 하겠습니다.

In [ ]:
housing['Address'].head()

# IsApt - New Feature

집 가격하면 대표적으로 아파트와 주택인지 나눌 수 있는데, 미국 주소는 다행히 'Apt.'이라는 구분으로 아파트를 구별합니다. 그래서 새로운 피쳐를 만들어주고 아파트인지 아닌지에 대한 여부를 저장하도록 하겠습니다.

In [ ]:
new_features = ['apt']

In [ ]:
aaa =housing['Address'].values

In [ ]:
#housing['Address'].loc[housing['Address'].contains(pat='Apt')]
housing_series = pd.Series(housing['Address'])
Apt_index = housing_series[housing_series.str.contains('Apt')].index
print(Apt_index[:10])

In [ ]:
housing['IsApt']=0
housing['IsApt'].iloc[Apt_index] = 1

아파트면 1 아니면 0으로 했습니다.

# ST_INitial - New Feature 

미국 연방을 분류하기위한 주소라고 함<a href="https://ko.wikipedia.org/wiki/%EB%AF%B8%EA%B5%AD_%EC%9A%B0%ED%8E%B8_%EC%95%BD%EC%9E%90">링크</a>

서울 구로 등등 도시들의 약자라고 합니다. 50개의 주가 있는데, 이 것들에 대한 약자로써 저는 여기에 대해서 원핫 인코딩을 시켜서 주의 약자 이름으로 새로운 피쳐들을 만드려고 합니다.

In [ ]:
ST_INitial = ['AK','AL','AR','AZ','CA','CO','CT','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY']



In [ ]:

for st in ST_INitial:
    housing[st]=0
    tmp_st_index = housing_series[housing_series.str.contains(st)].index
    housing[st].iloc[tmp_st_index] =1
    

In [ ]:
housing = housing.drop(['Address'],axis=1)

주소 피쳐에서 아파트냐 아니냐 주가 어디인지 다 알아냈으니 이제 스트링의 주소피쳐는 지워줍니다.

# Train and predict

In [ ]:
y_label = housing['Price']
housing = housing.drop(['Price'],axis=1)

집 값 즉,레이블을 분리해줍니다.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

<h3>Model selection</h3>

* 정확한 가격을 예측해야하는데 보통 이런 문제는 regression을 많이 사용합니다. 
* 가장 기본적인 1차함수 형태의 lienar regression을 사용할 것입니다. y= b0+ b1x+b2x2+b3x3 ... 형태 입니다.
* 여러개의 랜덤성을 지닌 DT(Decision Tree)을 엮어 앙상블을 한 랜덤 포레스트 모델을 사용하였습니다.
* DT는 계층구조로 이루어진 노드와 에지들의 집합으로 마치 스무고개를 하듯이 들어온 피쳐들의 브런치들로 나누는 것입니다.

* 이번모델에서 평가기준은 mse, mae ,rmse가 될것이구요 
* 각각 레이블에서 떨어진 거리의 제곱의 합, 떨어진 절대값의 합, mse에 루트를 씌운 값이 되겠습니다.
* 즉 3,4의 acc 와는 다르게 적을수록 좋습니다.

In [ ]:
rmse_res_list = []
mse_res_list = []
mae_res_list = []
model_list = ['lin_reg','rf_reg']

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(housing,y_label)
lin_predictions = lin_reg.predict(housing)
lin_mse = mean_squared_error(y_label,lin_predictions)
lin_mae = mean_absolute_error(y_label,lin_predictions)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
print("root mean_squared_error :",lin_rmse)

print("mean_squared_error : ",lin_mse)

print("mean_absolute_error :",lin_mae)
rmse_res_list.append(lin_rmse)
mse_res_list.append(lin_mse)
mae_res_list.append(lin_mae)

In [ ]:
param_grid = [
    {'n_estimators': [50,100],'max_features' : [8,20]},
    {'bootstrap':[False],'max_depth':[5,10]},
]
forest_reg = RandomForestRegressor()
rf_grid_search = GridSearchCV(forest_reg,param_grid,cv=5,
                          scoring='neg_mean_squared_error',return_train_score=True)

rf_grid_search.fit(housing,y_label)

In [ ]:
rf_cvres = rf_grid_search.cv_results_
for mean_score, params in zip(rf_cvres["mean_test_score"],rf_cvres["params"]):
    print(np.sqrt(-mean_score),params)

In [ ]:
rf_final_res = rf_grid_search.best_estimator_.predict(housing)
print("root mean_squared_error :",np.sqrt(mean_squared_error(rf_final_res,y_label)))
print("mean_squared_error : ", mean_squared_error(rf_final_res,y_label))
print("mean_absolute_error :",mean_absolute_error(rf_final_res,y_label))
rmse_res_list.append(np.sqrt(mean_squared_error(rf_final_res,y_label)))
mse_res_list.append(mean_squared_error(rf_final_res,y_label))
mae_res_list.append(mean_absolute_error(rf_final_res,y_label))

In [ ]:
from pandas import Series
fig,ax = plt.subplots(2,2,figsize=(9,5))
plt.figure(figsize=(8, 8))
sns.barplot(model_list,rmse_res_list,ax=ax[0][0])
ax[0][0].set_title("rmse")
sns.barplot(model_list,mse_res_list,ax=ax[0][1])
ax[0][1].set_title("mse")
sns.barplot(model_list,mae_res_list,ax=ax[1][0])
ax[1][0].set_title("mae")
plt.show()

In [ ]:
mae_res_list

# 성능확인

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].scatter(y_label,rf_final_res);
ax[0].set_title("random forest")
ax[1].scatter(y_label,lin_predictions);
ax[1].set_title("linear regression")

역시 여러 앙상블 기법등을 쎃은 RF 모델이 일반저인 linear regression모델을 압도했습니다.

# RF - Feature importance

linear regression은 단순 방정식이라 feature의 importance 라는게 없는데 RF 모델은 랜덤성과 앙상블이 있는 모델인만큼 Feature importance로 어떤 변수가 predict에 영향을 미쳤는지 확인 할수 있습니다.

In [ ]:
feature_importance = rf_grid_search.best_estimator_.feature_importances_
Series_feat_imp = Series(feature_importance, index=housing.columns)

In [ ]:
print(Series_feat_imp)

역시 집값이 가장 큰 영향을 미쳤고, 새로만든 원핫 피쳐들도 어느정도 영향은 있었습니다 그리고 새로만든 피쳐인 아파트이냐 아니냐의 피쳐가 생각보다 많은 영향을 미친 것을 알수 있습니다.